In [43]:
import argparse
import copy
import gzip
import heapq
import itertools
import os
import pickle
from collections import defaultdict
from itertools import count
import matplotlib.pyplot as plt

import numpy as np
from scipy.stats import norm
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.distributions.categorical import Categorical

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import ipywidgets as widgets
from ipywidgets import interact


In [44]:
_dev = [torch.device('cpu')]
# tf = lambda x: torch.FloatTensor(x).to(_dev[0])
# tl = lambda x: torch.LongTensor(x).to(_dev[0])
tf = lambda x: torch.FloatTensor(np.array(x)).to(_dev[0])  # Convert to numpy array first
tl = lambda x: torch.LongTensor(np.array(x)).to(_dev[0])

def set_device(dev):
    _dev[0] = dev 

def func_corners(x):
    ax = abs(x)
    return (ax > 0.5).prod(-1) * 0.5 + ((ax < 0.8) * (ax > 0.6)).prod(-1) * 2 + 1e-1



# Define the sigmoid function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Define the dynamical system for the n-node system with sigmoid
def node_system_with_sigmoid(x, t, coord):
    matrix_dim = len(x)
    M_tilde = np.reshape(coord, (matrix_dim, matrix_dim))
    z = M_tilde.dot(x)  # Compute M_tilde * x
    sigmoid_z = sigmoid(z)
    dxdt = sigmoid_z - x  # Compute the derivative   
    return dxdt

# Calculate reward given the weights
def reward_oscillator(coord, ndim):
    delta = 0.0001  # 0.0001
    matrix_dim = int(np.sqrt(ndim))
    x0 = np.linspace(0, 1, matrix_dim, endpoint=False)  # Initial conditions
    t = np.linspace(0, 20, 200)  # Define the time points
    sol = odeint(node_system_with_sigmoid, x0, t, args=(coord, ))
    
    # Calculate the total number of sharp peaks across all time series
    total_peaks = 0
    for i in range(matrix_dim):  # Loop through each time series x1, x2, ..., xn
        x_i = sol[:, i]
        dx_i = np.diff(x_i)  # First derivative approximation
        peaks = 0
        for j in range(1, len(dx_i)):
            if dx_i[j-1] > 0 and dx_i[j] < 0:  # Detect a peak
                sharpness = x_i[j] - (x_i[j-1] + x_i[j+1]) / 2
                if sharpness > delta:  # Check if the peak is sharp
                    peaks += 1
        total_peaks += peaks  # Add the number of sharp peaks for this time series
    
    if total_peaks == 0:
        return 2.5e-5  # see log_reg_c
    else:
        return min(total_peaks**5, 14**5)  # number of peaks   


In [45]:

class GridEnv:

    def __init__(self, horizon, ndim=2, multiplier=2, func=None):
        self.horizon = horizon
        self.ndim = ndim
        self.multiplier = multiplier
        self.func = func   # Sets the reward function.
        self._true_density = None
 
    def obs(self, s=None):
        """
        Returns a one-hot encoded observation of the current state.
        The observation is a flattened vector representing the agent's position in the grid.
        """
        s = np.int32(self._state if s is None else s)
        z = np.zeros((self.horizon * self.ndim), dtype=np.float32)
        z[np.arange(len(s)) * self.horizon + s] = 1 
        return z    # one-hot agent's current position in the grid.
    
    def s2x(self, s):
        """
        Transform the grid of indices (state s) to spherical coordinates and then calculate the cartesian coordinates
        (x_1, x_2, ..., x_n) for n dimensions.
        
        s[0]: radial distance (r)
        s[1:n-1]: polar angles (theta_1, theta_2, ..., theta_{n-2})
        s[n-1]: azimuthal angle (phi)
        
        multiplier: scalar value to multiply the state s (default is 1)
        """
        # Apply the multiplier to the state s
        s = s * self.multiplier
        
        # Initialize the radius (r) from the first component of the state vector
        r = s[0]
    
        # Initialize an array to hold the Cartesian coordinates
        x = np.zeros(self.ndim)
    
        # Constants or parameters (assuming self.horizon refers to the number of steps in the grid)
        horizon = self.horizon
        
        # Calculate the spherical to Cartesian conversion
        product = r
        for i in range(1, self.ndim):
            if i == self.ndim - 1:
                # The last angle (phi) ranges from 0 to 2π
                phi = s[i] * 2 * np.pi / horizon
                x[i - 1] = product * np.cos(phi)
                x[i] = product * np.sin(phi)
            else:
                # The other angles (theta) range from 0 to π
                theta = s[i] * np.pi / horizon
                x[i - 1] = product * np.sin(theta)
                product *= np.cos(theta)
    
        return x
    
    def reset(self):
        """
        Resets the environment to the initial state.
        """
        self._state = np.int32([0] * self.ndim)   # start position (0,0...)
        self._step = 0
        return self.obs(), self.func(self.s2x(self._state), self.ndim), self._state

    def parent_transitions(self, s, used_stop_action):
        """
        Determines the parent states and corresponding actions that could have led to the current state.
        
        Parameters:
        - s: The current state.
        - used_stop_action: A boolean indicating if the stop action was used.
        
        Returns:
        - A list of possible parent states (one-hot encoded).
        - A list of corresponding actions.
        """
        if used_stop_action:
            return [self.obs(s)], [self.ndim]
            
        parents = []
        actions = []
        for i in range(self.ndim):
            if s[i] > 0:
                sp = s.copy()  # s + 0
                sp[i] -= 1
                if sp.max() == self.horizon - 1:  # Can't have a terminal parent
                    continue
                parents.append(self.obs(sp))  # Generate observation for parent state
                actions.append(i)
        return parents, actions

    
    def step(self, a):
        """
        Updates the environment's state based on the action `a` and 
        returns the new observation, reward, done signal, and new state.
        """
        s = self._state.copy()
        if a < self.ndim:
            s[a] += 1
        
        done = s.max() >= self.horizon - 1 or a == self.ndim
        self._state = s  # Update the internal state
        self._step += 1  # Increment step counter
        
        return self.obs(), 0 if not done else self.func(self.s2x(s), self.ndim), done, s



class ReplayBuffer:
    def __init__(self, args, env):
        self.buf = []
        self.strat = args.replay_strategy
        self.sample_size = args.replay_sample_size
        self.bufsize = args.replay_buf_size
        self.env = env

    def add(self, x, r_x):
        if self.strat == 'top_k':
            if len(self.buf) < self.bufsize or r_x > self.buf[0][0]:
                self.buf = sorted(self.buf + [(r_x, x)])[-self.bufsize:]

    def sample(self):
        if not len(self.buf):
            return []
        idxs = np.random.randint(0, len(self.buf), self.sample_size)
        return sum([self.generate_backward(*self.buf[i]) for i in idxs], [])  # Samples from the buffer and generates trajectories backward.

    def generate_backward(self, r, s0):
        s = np.int8(s0)
        os0 = self.env.obs(s)
        # If s0 is a forced-terminal state, the the action that leads
        # to it is s0.argmax() which .parents finds, but if it isn't,
        # we must indicate that the agent ended the trajectory with
        # the stop action
        used_stop_action = s.max() < self.env.horizon - 1
        done = True
        # Now we work backward from that last transition
        traj = []
        while s.sum() > 0:
            parents, actions = self.env.parent_transitions(s, used_stop_action)
            # add the transition
            traj.append([tf(i) for i in (parents, actions, [r], [self.env.obs(s)], [done])])
            # Then randomly choose a parent state
            if not used_stop_action:
                i = np.random.randint(0, len(parents))
                a = actions[i]
                s[a] -= 1
            # Values for intermediary trajectory states:
            used_stop_action = False
            done = False
            r = 0
        return traj  # Generates a trajectory by working backward from a terminal state.

def make_mlp(l, act=nn.LeakyReLU(), tail=[]):
    return nn.Sequential(*(sum(
        [[nn.Linear(i, o)] + ([act] if n < len(l)-2 else [])
         for n, (i, o) in enumerate(zip(l, l[1:]))], []) + tail))
    
class FlowNetAgent:
    def __init__(self, args, envs):
        self.model = make_mlp([args.horizon * args.ndim] +
                              [args.n_hid] * args.n_layers +
                              [args.ndim+1])
        self.model.to(args.dev)
        self.target = copy.deepcopy(self.model)
        self.envs = envs
        self.ndim = args.ndim
        self.tau = args.bootstrap_tau
        self.replay = ReplayBuffer(args, envs[0])
        self.log_reg_c = args.log_reg_c
        
        # to store training data
        self.trn_all_losses = []
        self.trn_all_visited_done = []


    def parameters(self):
        return self.model.parameters()

    def sample_many(self, mbsize, all_visited_done):
        """Collects transition data from multiple parallel trajectories."""
        batch = []  # store transitions.
        batch += self.replay.sample()
        s = tf([i.reset()[0] for i in self.envs])
        done = [False] * mbsize
        while not all(done):
            # Note to self: this is ugly, ugly code
            with torch.no_grad():
                acts = Categorical(logits=self.model(s)).sample()   # Samples actions based on model's logits.
            step = [i.step(a) for i,a in zip([e for d, e in zip(done, self.envs) if not d], acts)]
            p_a = [self.envs[0].parent_transitions(sp_state, a == self.ndim)
                   for a, (sp, r, done, sp_state) in zip(acts, step)]
            batch += [[tf(i) for i in (p, a, [r], [sp], [d])]
                      for (p, a), (sp, r, d, _) in zip(p_a, step)]
            c = count(0)
            m = {j:next(c) for j in range(mbsize) if not done[j]}
            done = [bool(d or step[m[i]][2]) for i, d in enumerate(done)]
            s = tf([i[0] for i in step if not i[2]])
            for (_, r, d, sp) in step:
                if d:
                    all_visited_done.append((tuple(sp), r))  # (state, reward) pairs
                    self.replay.add(tuple(sp), r) 
        return batch  # it returns a batch of collected transitions for training. {parents, actions, reward, next_state, done}

    def sample_one_traj(self):
        """
        Samples a single trajectory and returns it.
        """
        traj = []
        env = self.envs[0]
        traj.append([[], [], 0, np.int32([0] * self.ndim), False])
        
        s = tf(env.reset()[0])
        done = False
        while not done:
            with torch.no_grad():
                logits = self.model(s.unsqueeze(0))
                action_dist = Categorical(logits=logits)
                a = action_dist.sample().item()
            sp, r, done_flag, sp_state = env.step(a)
            parent_states, parent_actions = env.parent_transitions(sp_state, a == self.ndim)
            traj.append([parent_states, parent_actions, r, sp_state, done_flag])
            
            s = tf(sp)
            done = done_flag
        return traj

    def learn_from(self, it, batch):
        loginf = tf([1000])
        batch_idxs = tl(sum([[i]*len(parents) for i, (parents,_,_,_,_) in enumerate(batch)], []))
        parents, actions, r, sp, done = map(torch.cat, zip(*batch))
        parents_Qsa = self.model(parents)[torch.arange(parents.shape[0]), actions.long()]
        in_flow = torch.log(self.log_reg_c + torch.zeros((sp.shape[0],))
                            .index_add_(0, batch_idxs, torch.exp(parents_Qsa)))
        if self.tau > 0:
            with torch.no_grad(): next_q = self.target(sp)
        else:
            next_q = self.model(sp)
        next_qd = next_q * (1-done).unsqueeze(1) + done.unsqueeze(1) * (-loginf)
        out_flow = torch.logsumexp(torch.cat([torch.log(self.log_reg_c + r)[:, None], next_qd], 1), 1)
        
        term_loss = ((in_flow - out_flow) * done).pow(2).sum() / (done.sum() + 1e-20)
        flow_loss = ((in_flow - out_flow) * (1-done)).pow(2).sum() / ((1-done).sum() + 1e-20)
        
        # loss = (in_flow - out_flow).pow(2).mean()
        leaf_coef = 10
        loss = term_loss * leaf_coef + flow_loss

        if self.tau > 0:
            for a,b in zip(self.model.parameters(), self.target.parameters()):
                b.data.mul_(1-self.tau).add_(self.tau*a)

        return loss, term_loss, flow_loss    




In [46]:
# Training 

def make_opt(params, args):
    params = list(params)
    if not len(params):
        return None
    if args.opt == 'adam':
        opt = torch.optim.Adam(params, args.learning_rate,
                               betas=(args.adam_beta1, args.adam_beta2))
    elif args.opt == 'msgd':
        opt = torch.optim.SGD(params, args.learning_rate, momentum=args.momentum)
    return opt

def compute_empirical_reward_distribution(visited):
    if not len(visited):
        return {}
    reward_hist = defaultdict(int)
    for _, reward in visited:
        reward_hist[reward] += 1
    total_visits = sum(reward_hist.values())
    empirical_distribution = {reward: count / total_visits for reward, count in reward_hist.items()}
    return empirical_distribution



all_losses = []
all_visited_done = []

def main(args):
    args.dev = torch.device(args.device)
    set_device(args.dev)
    f = {'default': None,
         'corners': func_corners,
         'oscillator': reward_oscillator,
    }[args.func]
    
    env = GridEnv(args.horizon, args.ndim, multiplier=args.multiplier, func=f)
    envs = [GridEnv(args.horizon, args.ndim, multiplier=args.multiplier, func=f)
            for i in range(args.mbsize)] 
    ndim = args.ndim
    nnode = args.nnode

    if args.method == 'flownet':
        agent = FlowNetAgent(args, envs)
    elif args.method == 'mcmc':
        agent = MHAgent(args, envs)
    elif args.method == 'random_traj':
        agent = RandomTrajAgent(args, envs)

    opt = make_opt(agent.parameters(), args)

        
    
    # Log file setup
    root = args.save_path 
    log_file_path = os.path.join(root, f'trn-out-{args.nnode}-node.log')
    os.makedirs(os.path.dirname(log_file_path), exist_ok=True)
    with open(log_file_path, 'w') as log_file:
    
        # Training Loop Setup
        
        ttsr = max(int(args.train_to_sample_ratio), 1) # train to sample ratio
        sttr = max(int(1/args.train_to_sample_ratio), 1) # sample to train ratio
        
        for i in tqdm(range(args.n_train_steps+1), disable=not args.progress):
            data = []  # a list of transitions from a batch of trajectories
            for j in range(sttr):
                """Agent samples trajectories for training."""
                data += agent.sample_many(args.mbsize, all_visited_done)   
            for j in range(ttsr):
                """Agent updates its model using the sampled data."""
                losses = agent.learn_from(i * ttsr + j, data) # returns (opt loss, *metrics)
                if losses is not None:
                    losses[0].backward(retain_graph=(not i % 50))
                    if args.clip_grad_norm > 0:
                        torch.nn.utils.clip_grad_norm_(agent.parameters(),
                                                       args.clip_grad_norm)
                    opt.step()
                    opt.zero_grad()
                    all_losses.append([i.item() for i in losses])
        
            # Log empirical reward every 100 iterations
            if not i % 100:
                empirical_distribution = compute_empirical_reward_distribution(all_visited_done[-args.num_empirical_loss:])
                print('Partial Empirical Reward Distribution:', empirical_distribution)
                log_file.write(f'Partial Empirical Reward Distribution: {empirical_distribution}\n')
                log_file.flush()  # Ensure data is written to the log file
                        
            # Save the agent and model every 1000 iterations
            if not i % 1000:
                # Update agent with current all_losses and all_visited_done
                agent.trn_all_losses = all_losses.copy()
                agent.trn_all_visited_done = all_visited_done.copy()
                
                # Save the entire agent
                agent_save_path = os.path.join(root, f"agent_checkpoint_{i}.pkl.gz")  
                with gzip.open(agent_save_path, 'wb') as f: 
                    pickle.dump(agent, f)
                print(f"Agent checkpoint saved at iteration {i} in {nnode}.")
                log_file.write(f"Agent checkpoint saved at iteration {i} in {nnode}.\n")
                log_file.flush() 
            
                # Save the agent's model separately
                model_save_path = os.path.join(root, f"model_checkpoint_{i}.pkl.gz") 
                with gzip.open(model_save_path, 'wb') as f:  
                    pickle.dump(agent.model, f)
                print(f"Model checkpoint saved at iteration {i} in {nnode}.")
                log_file.write(f"Model checkpoint saved at iteration {i} in {nnode}.\n")
                log_file.flush() 


In [ ]:
class Args:
    save_path = 'results-v3/7-node-v2'
    device = 'cpu'
    progress = True  
    method = 'flownet'
    learning_rate = 5e-4
    opt = 'adam'
    adam_beta1 = 0.9
    adam_beta2 = 0.999
    momentum = 0.9  # SGD with momentum
    mbsize = 8  # number of parallel environments (trajectories) are collected by one agent (One Agent's model is shared in Many Environments). 
    train_to_sample_ratio = 1.0  # determines how many times the agent should update its model (train) for each set of data it collects from the environment. 
    clip_grad_norm = 0.
    n_hid = 256  # number of hidden units in each hidden layer
    n_layers = 3
    n_train_steps = 30000 
    num_empirical_loss = 200000  # number of samples used to compute the empirical distribution loss during evaluation.
    
    
    # Env
    func = 'oscillator'
    horizon = 8  # 4*2
    nnode = 7
    ndim = nnode*nnode 
    multiplier = 3
    
    # Flownet
    bootstrap_tau = 0.0  # no bootstrapping,target network isn't being updated gradually but possibly replaced entirely at some point.
    replay_strategy = 'top_k'  # 'top_k' or 'none'
    replay_sample_size = 5  # number of experiences to sample from the replay buffer at each update step.
    replay_buf_size = 100  #  size of the replay buffer, which stores past experiences for the agent to learn from.
    log_reg_c = 2.5e-5



args = Args()
torch.set_num_threads(200)
main(args)



  0%|          | 1/30001 [00:00<2:39:14,  3.14it/s]

Partial Empirical Reward Distribution: {2.5e-05: 1.0}
Agent checkpoint saved at iteration 0 in 7.
Model checkpoint saved at iteration 0 in 7.


  0%|          | 101/30001 [00:42<2:22:18,  3.50it/s]

Partial Empirical Reward Distribution: {2.5e-05: 0.9170792079207921, 1: 0.07920792079207921, 32: 0.0037128712871287127}


  1%|          | 201/30001 [01:04<1:50:18,  4.50it/s]

...

 20%|██        | 6103/30001 [13:39<39:07, 10.18it/s]  

Partial Empirical Reward Distribution: {2.5e-05: 0.9675872807736436, 1: 0.030220455663006063, 32: 0.0020283560072119323, 1024: 2.0488444517292246e-05, 243: 0.00014341911162104573}


 21%|██        | 6203/30001 [13:48<39:28, 10.05it/s]

Partial Empirical Reward Distribution: {2.5e-05: 0.9681099822609257, 1: 0.02973310756329624, 32: 0.0019956458635703917, 1024: 2.0158039025963555e-05, 243: 0.00014110627318174489}


 21%|██        | 6265/30001 [13:54<36:09, 10.94it/s]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming all_losses, all_visited, empirical_distrib_losses are lists of lists or arrays

# Plot the losses
if all_losses:
    all_losses_array = np.array(all_losses)
    plt.figure(figsize=(12, 6))
    for i in range(all_losses_array.shape[1]):
        plt.plot(all_losses_array[:, i], label=f'Loss {i+1}')
    plt.xlabel('Training Step')
    plt.ylabel('Loss')
    plt.title('Training Losses Over Time')
    plt.legend()
    plt.show()


In [ ]:
# Sort visited states by reward in descending order
visited_sorted_by_reward = sorted(all_visited_done, key=lambda x: x[1], reverse=True)

# Sample the top 10 states with the highest rewards
top_states = visited_sorted_by_reward[:20]

print("Top 10 States with Highest Rewards:")
for i, (state, reward) in enumerate(top_states):
    print(f"State {i+1}: {state}, Reward: {reward}")


In [ ]:
import pickle
import gzip
import os

args = Args()

env = GridEnv(args.horizon, args.ndim, func=f)

agent_path = "/home/dannyhuang/gflownet/frontline/results-v3/7-node/agent_checkpoint_30000.pkl.gz"
print(agent_path)

# Load the trained_agent object from the file
try:
    with gzip.open(agent_path, 'rb') as f:
        trained_agent = pickle.load(f)
    print(trained_agent)
except FileNotFoundError:
    print(f"Error: The file {agent_path} was not found.")
except Exception as e:
    print(f"An error occurred while loading the file: {str(e)}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import ipywidgets as widgets
from ipywidgets import interact

delta = 0.0001

# Define the sigmoid function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Define the dynamical system for the n-node system with sigmoid
def n_node_system_with_sigmoid(x, t, *weights):
    nnode = int(np.sqrt(len(weights)))
    M_tilde = np.array(weights).reshape((nnode, nnode))
    
    # Compute M_tilde * x
    z = M_tilde.dot(x)
    
    # Apply sigmoid
    sigmoid_z = sigmoid(z)
    
    # Compute the derivative
    dxdt = sigmoid_z - x
    return dxdt

# Function to update plot and calculate reward
def update_plot(*weights):
    nnode = int(np.sqrt(len(weights)))
    ndim = nnode * nnode

    # Initial conditions
    x0 = np.linspace(0, 1, nnode, endpoint=False)

    # Define the time points
    t = np.linspace(0, 20, 200)

    # Solve ODE
    sol = odeint(n_node_system_with_sigmoid, x0, t, args=tuple(weights))
    
    # Clear current plot
    plt.clf()
    
    # Plot the results
    for i in range(nnode):
        plt.plot(t, sol[:, i], label=f'$x_{i+1}$')
    plt.xlabel('Time')
    plt.ylabel('Concentration')
    plt.title(f'{nnode}-Node System Dynamics with Sigmoid')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    # Calculate reward based on the number of sharp peaks
    reward = calculate_reward(sol)
    print(f"Reward: {reward}")

# Function to calculate reward based on the number of sharp peaks
def calculate_reward(sol, delta=delta):
    reward = 0
    for i in range(sol.shape[1]):  # Loop through each time series x1, x2, ..., xn
        x_i = sol[:, i]
        dx_i = np.diff(x_i)  # First derivative approximation
        peaks = 0
        for j in range(1, len(dx_i)):
            if dx_i[j-1] > 0 and dx_i[j] < 0:  # Detect a peak
                sharpness = x_i[j] - (x_i[j-1] + x_i[j+1]) / 2
                if sharpness > delta:  # Check if the peak is sharp
                    peaks += 1
        reward += peaks  # Add the number of sharp peaks as reward
    return reward


# # Example usage
# nnode = 5  # Change this to set the number of nodes
# ndim = nnode * nnode
# test_weights = np.ones(ndim)
# update_plot(*test_weights)

# test_state = [3, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
# 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
# test_weight = env.s2x(np.int32(test_state))
# update_plot(*test_weight)




# Create sliders for each weight
import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display

# Initialize weights based on the test_state
initial_weights = env.s2x(np.int32([3, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]))

# Create sliders for all 49 weights
sliders = [widgets.FloatSlider(value=initial_weights[i], min=-15, max=15, step=0.1, description=f'Weight {i+1}') for i in range(49)]

# Create an output widget to display the plot
out = widgets.Output()

# Define the update function
def update_plot_with_sliders(**w):
    with out:
        out.clear_output(wait=True)
        new_weights = list(w.values())
        update_plot(*new_weights)

# Create the interactive plot
interactive_plot = interactive(update_plot_with_sliders, **{f'w{i}': slider for i, slider in enumerate(sliders)})

# Display the sliders and the plot
display(widgets.VBox([interactive_plot, out]))

# Initial plot update
update_plot_with_sliders(**{f'w{i}': initial_weights[i] for i in range(49)})

In [ ]:
initial_weights